In [1]:
import pandas as pd
import geopandas as gdp

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
train_station_gdp = gdp.read_file("../data/raw/train_station/ll_gda2020/esrishape/whole_of_dataset/victoria/TRANSPORT/VIC_RAILWAY_STATIONS.shp")
train_station_gdp

,STATION,STATUS,STOPID_MTR,STID_VLINE,PREMIUM,METRO,VLINE,INTERSTATE,STOP_ZONE,COMMENTS,geometry
0,Dandenong,Active,19888,22250,Yes,Yes,Yes,No,"Reg, 2",Metro Interchange,POINT (145.20964 -37.98992)
1,Darebin,Active,19932,None,No,Yes,No,No,1,None,POINT (145.03863 -37.77480)
2,Darling,Active,19837,None,Yes,Yes,No,No,"1, 2",None,POINT (145.06268 -37.86867)
3,Deer Park,Active,None,19982,No,No,Yes,No,"Reg, 2",None,POINT (144.77122 -37.77750)
4,Dennis,Active,19929,None,No,Yes,No,No,1,None,POINT (145.00871 -37.77918)
...,...,...,...,...,...,...,...,...,...,...,...
311,Cranbourne,Active,19886,None,Yes,Yes,No,No,2,Metro Terminus,POINT (145.28001 -38.09916)
312,Creswick,Active,None,44951,No,No,Yes,No,"Reg, 9",None,POINT (143.88816 -37.42448)
313,Crib Point,Active,19828,None,No,Yes,No,No,2,None,POINT (145.20398 -38.36593)
314,Croxton,Active,20015,None,No,Yes,No,No,1,None,POINT (144.99710 -37.76384)


In [5]:
train_station_gdp['coord'] = [[train_station_gdp['geometry'].y[i], train_station_gdp['geometry'].x[i]] for i in range(0, len(train_station_gdp['geometry'].x))]

In [6]:
from geopy.distance import geodesic
def distance(p1, p2):
    return geodesic(tuple(p1), tuple(p2)).m

In [7]:
properties = pd.read_json("../data/raw/property_raw.json")
properties = properties.transpose()
properties = properties.reset_index(drop=True)

In [ ]:
from collections import defaultdict
station_properties = defaultdict(list)
cls_stations = []
j = 0
for coor in properties["coordinates"]:
    cls_station_id = -1
    min_dist = 10000000000000000
    i = 0
    for station in train_station_gdp['coord']:
        if distance(coor, station) <= min_dist:
            min_dist = distance(coor, station)
            cls_station_id = i
        i += 1
    cls_stations.append(cls_station_id)
    station_properties[cls_station_id].append(j)
    j += 1
    
    

In [9]:
properties["closest station"] = pd.Series(data = cls_stations)
properties

,name,cost_text,coordinates,rooms,type,closest station
0,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,"[-37.8318086, 144.9710272]","[3 Beds, 2 Baths, 2 Parking]",Apartment / Unit / Flat,30
1,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,"[-37.816228, 144.9532465]","[1 Bed, 1 Bath, 1 Parking]",Apartment / Unit / Flat,176
2,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,"[-37.8121026, 144.9581291]","[1 Bed, 1 Bath]",Apartment / Unit / Flat,28
3,3/27 Flinders Lane Melbourne VIC 3000,$420 per week,"[-37.8147317, 144.9733655]","[1 Bed, 1 Bath]",Apartment / Unit / Flat,140
4,611/408 Lonsdale Street Melbourne VIC 3000,$350,"[-37.8125979, 144.9604012]","[1 Bed, 1 Bath]",Apartment / Unit / Flat,28
...,...,...,...,...,...,...
15342,28 Beachcomber Drive Inverloch VIC 3996,$550 wk,"[-38.6412502, 145.7031282]","[4 Beds, 2 Baths]",House,181
15343,14 Inverloch Parade Inverloch VIC 3996,$420 wk,"[-38.6287143, 145.7358082]","[2 Beds, 1 Bath]",House,181
15344,10 Hopetoun Street Inverloch VIC 3996,$370 wk,"[-38.6341243, 145.7306862]","[2 Beds, 1 Bath, 1 Parking]",House,181
15345,Inverloch VIC 3996,$475 wk,"[-38.6314613, 145.7293638]","[3 Beds, 2 Baths, 1 Parking]",House,181


In [10]:
import re
import openrouteservice as ors
import folium
import time
client = ors.Client(key='your key')

In [11]:
# This block of code finds out the distance of the properties to CBD Melbourne
error_station = []
result = {}
for station in station_properties.keys():
    station_coord = list(reversed(train_station_gdp.iloc[station]['coord']))
    prop_coords = [list(reversed(properties.iloc[prop]['coordinates'])) for prop in station_properties[station]]
    coordinates = [station_coord] + prop_coords
    [[float("{:.7f}".format(y))] for [x,y] in coordinates]
    print(f"Start station {station} request request size {len(coordinates)}")
    try:
        matrix = client.distance_matrix(
            locations=coordinates,
            destinations = [0],
            profile='foot-walking',
            metrics=['distance'],
            validate=False,
        )
    except:
        error_station.append(station)
    time.sleep(1.5)
    print(f"end {station}th request")
    curr = [dist[0] for dist in matrix['distances'][1:]]
    result[station] = curr

Start station 30 request request size 272
end 30th request
Start station 176 request request size 392
end 176th request
Start station 28 request request size 231
end 28th request
Start station 140 request request size 52
end 140th request
Start station 94 request request size 406
end 94th request
Start station 175 request request size 215
end 175th request
Start station 144 request request size 148
end 144th request
Start station 129 request request size 84
end 129th request
Start station 214 request request size 53
end 214th request
Start station 68 request request size 23
end 68th request
Start station 128 request request size 95
end 128th request
Start station 150 request request size 25
end 150th request
Start station 31 request request size 73
end 31th request
Start station 213 request request size 78
end 213th request
Start station 101 request request size 83
end 101th request
Start station 168 request request size 13
end 168th request
Start station 195 request request size 120
e

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/opt/anaconda3/envs/ads/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/opt/anaconda3/envs/ads/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/opt/anaconda3/envs/ads/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


end 183th request
Start station 142 request request size 49
end 142th request
Start station 66 request request size 60
end 66th request
Start station 286 request request size 49
end 286th request
Start station 111 request request size 51
end 111th request
Start station 98 request request size 22
end 98th request
Start station 306 request request size 45
end 306th request
Start station 26 request request size 25
end 26th request
Start station 267 request request size 35
end 267th request
Start station 45 request request size 44
end 45th request
Start station 200 request request size 18
end 200th request
Start station 227 request request size 32
end 227th request
Start station 162 request request size 76
end 162th request
Start station 171 request request size 20
end 171th request
Start station 160 request request size 49
end 160th request
Start station 67 request request size 74
end 67th request
Start station 203 request request size 66
end 203th request
Start station 161 request reques

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


end 99th request
Start station 132 request request size 39
end 132th request
Start station 314 request request size 43
end 314th request
Start station 145 request request size 55
end 145th request
Start station 148 request request size 68
end 148th request
Start station 310 request request size 118
end 310th request
Start station 6 request request size 11
end 6th request
Start station 9 request request size 85
end 9th request
Start station 308 request request size 34
end 308th request
Start station 52 request request size 78
end 52th request
Start station 304 request request size 25
end 304th request
Start station 216 request request size 17
end 216th request
Start station 4 request request size 22
end 4th request
Start station 25 request request size 57
end 25th request
Start station 192 request request size 44
end 192th request
Start station 65 request request size 38
end 65th request
Start station 1 request request size 34
end 1th request
Start station 273 request request size 42
en

/opt/anaconda3/envs/ads/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


end 73th request
Start station 191 request request size 65
end 191th request
Start station 79 request request size 34
end 79th request
Start station 21 request request size 89
end 21th request
Start station 231 request request size 38
end 231th request
Start station 251 request request size 40
end 251th request
Start station 56 request request size 89
end 56th request
Start station 12 request request size 23
end 12th request
Start station 88 request request size 83
end 88th request
Start station 210 request request size 75
end 210th request
Start station 105 request request size 31
end 105th request
Start station 46 request request size 31
end 46th request
Start station 5 request request size 15
end 5th request
Start station 20 request request size 38
end 20th request
Start station 64 request request size 4
end 64th request
Start station 42 request request size 95
end 42th request
Start station 260 request request size 108
end 260th request
Start station 291 request request size 51
end

In [30]:
for station in error_station:
    station_coord = list(reversed(train_station_gdp.iloc[station]['coord']))
    prop_coords = [list(reversed(properties.iloc[prop]['coordinates'])) for prop in station_properties[station]]
    coordinates = [station_coord] + prop_coords
    [[float("{:.7f}".format(y))] for [x,y] in coordinates]
    print(f"Start station {station} request request size {len(coordinates)}")
    try:
        matrix = client.distance_matrix(
            locations=coordinates,
            destinations = [0],
            profile='foot-walking',
            metrics=['distance'],
            validate=False,
        )
    except:
        error_station.append(station)
    time.sleep(1.5)
    print(f"end {station}th request")
    curr = [dist[0] for dist in matrix['distances'][1:]]
    result[station] = curr

Start station 238 request request size 161
end 238th request


In [15]:
result

{30: [1803.65,
  471.84,
  208.11,
  243.63,
  565.42,
  843.24,
  422.84,
  243.63,
  781.53,
  840.45,
  1603.08,
  468.42,
  363.75,
  363.75,
  491.35,
  363.75,
  363.75,
  363.75,
  243.63,
  363.75,
  492.69,
  247.73,
  414.43,
  861.12,
  243.63,
  491.35,
  455.67,
  877.68,
  838.2,
  312.45,
  468.42,
  243.63,
  414.43,
  793.13,
  839.04,
  638.51,
  638.51,
  877.57,
  565.42,
  312.45,
  363.75,
  363.75,
  363.75,
  666.65,
  904.57,
  898.75,
  638.51,
  1032.68,
  1603.08,
  1622.85,
  1622.85,
  1032.68,
  1065.93,
  1570.46,
  1464.46,
  1012.01,
  1509.53,
  667.97,
  1634.96,
  667.97,
  1464.46,
  1475.67,
  1509.53,
  1431.31,
  1441.22,
  1377.88,
  927.96,
  1245.55,
  1441.22,
  667.97,
  1603.03,
  1188.17,
  1012.01,
  1038.12,
  1371.75,
  1134.0,
  1012.01,
  1441.22,
  1104.43,
  1104.43,
  1238.04,
  1238.04,
  1245.55,
  1104.43,
  1027.99,
  1012.01,
  1570.46,
  1238.04,
  1164.39,
  1046.47,
  731.1,
  1252.4,
  1080.93,
  1238.04,
  1603.03,
  145

In [16]:
station_properties

defaultdict(list,
            {30: [0,
              5,
              13,
              21,
              25,
              38,
              43,
              50,
              51,
              52,
              57,
              103,
              104,
              111,
              114,
              125,
              126,
              133,
              139,
              154,
              162,
              172,
              179,
              202,
              223,
              262,
              269,
              279,
              299,
              311,
              312,
              320,
              336,
              338,
              343,
              344,
              347,
              348,
              353,
              360,
              442,
              488,
              516,
              550,
              555,
              586,
              589,
              707,
              709,
              739,
              751,
              786,
   

In [35]:
prop_dist = {}
for station in result.keys():
    props = station_properties[station]
    dists = result[station]
    for i in range(0, len(props)):
        prop_dist[props[i]] = dists[i]

In [32]:
lenth = 0
sum([len(x) for x in result.values()])

15347

In [37]:
len(prop_dist)

15347

In [40]:
prop_dist_data = [prop_dist[i] for i in range(0, len(prop_dist))]

In [41]:
prop_dist_data

[1803.65,
 371.68,
 248.5,
 404.26,
 525.08,
 471.84,
 433.14,
 161.64,
 371.68,
 650.24,
 650.24,
 433.14,
 544.71,
 208.11,
 575.87,
 281.38,
 644.84,
 652.25,
 579.93,
 520.64,
 447.55,
 243.63,
 655.86,
 334.71,
 605.72,
 565.42,
 550.36,
 2745.41,
 97.94,
 414.0,
 362.91,
 574.28,
 471.35,
 553.96,
 291.05,
 102.71,
 644.84,
 380.86,
 843.24,
 725.28,
 447.93,
 725.28,
 249.31,
 422.84,
 360.65,
 196.54,
 1433.24,
 460.08,
 2214.3,
 362.91,
 243.63,
 781.53,
 840.45,
 299.73,
 1664.71,
 273.72,
 102.71,
 1603.08,
 273.72,
 544.71,
 544.71,
 516.78,
 337.66,
 273.72,
 475.19,
 201.42,
 614.6,
 1353.9,
 459.02,
 550.36,
 327.65,
 520.64,
 150.83,
 362.91,
 333.47,
 362.91,
 493.4,
 298.63,
 317.0,
 525.08,
 248.5,
 357.84,
 56.1,
 410.05,
 614.6,
 367.34,
 585.76,
 442.79,
 404.93,
 564.74,
 325.7,
 570.84,
 333.47,
 471.97,
 382.09,
 322.01,
 644.84,
 333.47,
 56.1,
 505.34,
 56.1,
 97.94,
 290.44,
 468.42,
 363.75,
 505.99,
 447.55,
 629.25,
 629.25,
 414.0,
 335.03,
 363.75,
 614

In [46]:
properties["proximity to train station"] = pd.Series(data = prop_dist_data)

In [48]:
properties.to_csv("../data/raw/properties_train_proximity.csv")

In [32]:
# merge train distance to raw data
train_prox = pd.read_csv("../data/raw/properties_train_proximity.csv")
train_prox

,Unnamed: 0,name,cost_text,coordinates,rooms,type,closest station,proximity to train station
0,0,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,"[-37.8318086, 144.9710272]","['3 Beds', '2 Baths', '2 Parking']",Apartment / Unit / Flat,30,1803.65
1,1,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,"[-37.816228, 144.9532465]","['1 Bed', '1 Bath', '1 Parking']",Apartment / Unit / Flat,176,371.68
2,2,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,"[-37.8121026, 144.9581291]","['1 Bed', '1 Bath']",Apartment / Unit / Flat,28,248.50
3,3,3/27 Flinders Lane Melbourne VIC 3000,$420 per week,"[-37.8147317, 144.9733655]","['1 Bed', '1 Bath']",Apartment / Unit / Flat,140,404.26
4,4,611/408 Lonsdale Street Melbourne VIC 3000,$350,"[-37.8125979, 144.9604012]","['1 Bed', '1 Bath']",Apartment / Unit / Flat,28,525.08
...,...,...,...,...,...,...,...,...
15342,15342,28 Beachcomber Drive Inverloch VIC 3996,$550 wk,"[-38.6412502, 145.7031282]","['4 Beds', '2 Baths']",House,181,65073.96
15343,15343,14 Inverloch Parade Inverloch VIC 3996,$420 wk,"[-38.6287143, 145.7358082]","['2 Beds', '1 Bath']",House,181,69323.16
15344,15344,10 Hopetoun Street Inverloch VIC 3996,$370 wk,"[-38.6341243, 145.7306862]","['2 Beds', '1 Bath', '1 Parking']",House,181,68313.03
15345,15345,Inverloch VIC 3996,$475 wk,"[-38.6314613, 145.7293638]","['3 Beds', '2 Baths', '1 Parking']",House,181,68459.23


In [33]:
prop_raw = pd.read_json("../data/raw/property_raw.json")
prop_raw = prop_raw.transpose()
prop_raw = prop_raw.reset_index(drop=True)
prop_raw

,name,cost_text,coordinates,rooms,type
0,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,"[-37.8318086, 144.9710272]","[3 Beds, 2 Baths, 2 Parking]",Apartment / Unit / Flat
1,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,"[-37.816228, 144.9532465]","[1 Bed, 1 Bath, 1 Parking]",Apartment / Unit / Flat
2,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,"[-37.8121026, 144.9581291]","[1 Bed, 1 Bath]",Apartment / Unit / Flat
3,3/27 Flinders Lane Melbourne VIC 3000,$420 per week,"[-37.8147317, 144.9733655]","[1 Bed, 1 Bath]",Apartment / Unit / Flat
4,611/408 Lonsdale Street Melbourne VIC 3000,$350,"[-37.8125979, 144.9604012]","[1 Bed, 1 Bath]",Apartment / Unit / Flat
...,...,...,...,...,...
15342,28 Beachcomber Drive Inverloch VIC 3996,$550 wk,"[-38.6412502, 145.7031282]","[4 Beds, 2 Baths]",House
15343,14 Inverloch Parade Inverloch VIC 3996,$420 wk,"[-38.6287143, 145.7358082]","[2 Beds, 1 Bath]",House
15344,10 Hopetoun Street Inverloch VIC 3996,$370 wk,"[-38.6341243, 145.7306862]","[2 Beds, 1 Bath, 1 Parking]",House
15345,Inverloch VIC 3996,$475 wk,"[-38.6314613, 145.7293638]","[3 Beds, 2 Baths, 1 Parking]",House


In [34]:
prop_raw[['closest station', 'proximity to train station']] = train_prox[['closest station', 'proximity to train station']]

In [35]:
CBD_prox = pd.read_csv('../data/raw/CBD_distance.csv')
CBD_prox

,Unnamed: 0,name,CBD_distance
0,0,1901/368 St Kilda Road Melbourne VIC 3000,1702.19
1,1,1211/200 Spencer Street Melbourne VIC 3000,2267.13
2,2,1008/380 Little Lonsdale Street Melbourne VIC ...,1937.39
3,3,3/27 Flinders Lane Melbourne VIC 3000,790.13
4,4,611/408 Lonsdale Street Melbourne VIC 3000,1657.95
...,...,...,...
15342,15342,28 Beachcomber Drive Inverloch VIC 3996,146939.64
15343,15343,14 Inverloch Parade Inverloch VIC 3996,147493.50
15344,15344,10 Hopetoun Street Inverloch VIC 3996,147082.53
15345,15345,Inverloch VIC 3996,146682.64


In [38]:
# check if index have the same order
com_result = CBD_prox['name'].equals(other=prop_raw['name'])

In [41]:
prop_raw.to_csv('../data/raw/property_train_CBD.csv')